In [5]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import matplotlib.pyplot as plt
from transforms3d.euler import euler2mat
from mpl_toolkits.mplot3d import Axes3D
import pickle
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import torch
import torch.nn as nn

from Joint import *
from Viewer import *
from amc_parser import *

from AMCDataset import *


def data_loader_setup(ds:Dataset, config):
    indices = list(range(len(ds)))
    split   = int(np.floor(config['validation_split'] * len(ds)))

    if config['shuffle_dataset']:
        np.random.seed(config['random_seed'])
        np.random.shuffle(indices)

    train_indices, val_indices = indices[split:], indices[:split]

    ds_train = torch.utils.data.Subset(ds, train_indices)
    ds_valid = torch.utils.data.Subset(ds, val_indices)

    train_loader = torch.utils.data.DataLoader(
        ds_train, 
        batch_size=config['batch_size'], 
        shuffle=config['shuffle_dataset']
    )
    valid_loader = torch.utils.data.DataLoader(
        ds_valid, 
        batch_size=config['batch_size'], 
        shuffle=False
    )
    
    return train_loader, valid_loader


with open('./data/df_amc_full.pkl', 'rb') as f:
    df = pickle.load(f)

pygame 2.1.1.dev1 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
classes = {'walk', 'dance', 'jump', 'steps', 'acrobatics', 'interaction', 'getting up', 'pregnant', 'recreation', 'swimming', 
'soccer', 'basketball', 'skateboard', 'golf', 'run', 'motion', 'swing', 'salsa', 'charelston', 'everyday'}

class_assignment = {'walk': 'walk', 'steps': 'walk', 'dance':'dance', 'jump':'jump', 'acrobatics': 'acrobatics', 
'interaction': 'interaction', 'getting up':'stand up', 'pregnant': 'pregnant', 'swimming':'swimming', 'recreation':'recreation',
'soccer':'soccer', 'basketball': 'basketball', 'skateboard':'skateboard', 'golf':'golf', 'run':'run', 'motion':'motion',
'swing':'swing', 'salsa': 'dance 2', 'charelston': 'dance 2', 'everyday': 'everyday'
}
main_indices = {x:y for (x,y) in df.index}
desc_list = set()
for k in main_indices:
    desc = df.at[(k,main_indices[k]), 'id_desc'].lower()
    if desc != '':
        desc_list.add(desc.lower())
space = 20
desc_list_dict = {d: np.nan for d in desc_list}
for key in desc_list_dict:
    if any(x in key for x in classes):
        for c in classes:
            if c in key:
                #print(f"assining class {c} to {desc}")
                desc_list_dict[key] = class_assignment[c]
    #else:
        #print(f" {desc_list_dict[key]:<{space}} -> {key}")
df['label'] = np.nan
for key in desc_list_dict:
    df.loc[df.id_desc.str.lower() == key, 'label'] = desc_list_dict[key]
df.dropna(subset=['label'], inplace=True)
ds = AMCDataset(df)

In [26]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)

        self.linear = nn.Linear(hidden_size,config['num_classes'])
        self.softmax = nn.LogSoftmax(dim=1)
        

    def forward(self, x, h0):
        out, (h,c) = self.rnn(x,h0)
        
        h_out = self.linear(h)
        h_out = self.softmax(h_out)

        return out, h_out, (h,c)


In [27]:
actual_classes = {class_assignment[k] for k in classes}
class_indices = {c:idx for idx, c in enumerate(actual_classes)}
input_size = 93
max_seq_length = max(x.shape[0] for x in df.data)
hidden_size = 128
num_layers = 2
config = {}
config['random_seed'] = 1
config['shuffle_dataset'] = True
config['validation_split'] = 0.1
config['batch_size'] = 16
config['num_classes'] = len(actual_classes)
learning_rate = 0.001
train_loader, validation_loader = data_loader_setup(ds,config)

In [28]:
rnn = RNN(input_size, hidden_size, num_layers, len(actual_classes))
h = torch.zeros(num_layers, config['batch_size'], hidden_size)
c = torch.zeros(num_layers, config['batch_size'], hidden_size)
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [29]:
criterion = nn.NLLLoss()
rnn.train()
for X, (categories, _) in train_loader:
    target_categories = [class_indices[x] for x in categories]

    _, h_out, (h,c) = rnn(X.transpose(0,1).float(), (h,c))

    loss = criterion(h_out, torch.tensor(target_categories))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    break